<a href="https://colab.research.google.com/github/vishnuvardhan-jadava/BoW/blob/main/BoW_Fit_Transform_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
#importing libraries
import re
import pandas as pd
import sqlite3
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
con = sqlite3.connect('/content/drive/MyDrive/Dataset/database.sqlite')
df = pd.read_sql_query('select * from reviews where score <>3 limit 10',con)

In [5]:
df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...
6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,5,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...
7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,5,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...
8,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,5,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...
9,10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,5,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...


In [6]:
# functions to remove html tags, URL, punctuations, words with numbers and more then one space.
def clean_html_tags(x):
    """removes html tags"""
    return re.sub(r'<.*?>',r' ',x)
def clean_punctuation(x):
    """removes punctuations"""
    return re.sub(r'[^a-zA-Z0-9]+',r' ',x)
def remove_url(x):
    """removes URL"""
    return re.sub('http\S+',r' ',x)
def remove_words_with_numbers(x):
    """removes words with numbers"""
    return re.sub(r'[a-zA-Z]*[0-9]+[a-zA-Z]*',r' ',x)
def remove_more_than_one_space(x):
    return re.sub(r'[\s]{2,}',r' ',x)
def decontracted(phrase):
    """converts imformal words to formal"""
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def clean_text(text):
    """cleans the given text by removing html tags, URL, punctuations, words with numbers and more then one space"""
    text = decontracted(text)
    text = clean_html_tags(text)
    text = remove_url(text)
    text = clean_punctuation(text)
    text = remove_words_with_numbers(text)
    text = text.lower()
    text=remove_more_than_one_space(text)
    return text

In [7]:
df['cleaned_text'] = df['Text'].map(clean_text)

In [44]:
def fit_1(dataset):
  """returns vocab from the dataset"""
  uniq_words = set()
  if isinstance(dataset,(list)):
    for sent in dataset:
      for word in sent.split():
        if len(word)<2:
          continue
        uniq_words.add(word)
    uniq_words=sorted(uniq_words)
    vocab = {j:i for i,j in enumerate(uniq_words)}
    return vocab
  else:
    return 'pass a list'
        

In [12]:
vocab = fit_1(list(df['cleaned_text']))

In [45]:
from collections import Counter
from scipy.sparse import csr_matrix

def transform(dataset,vocab):
  """transform method, returns sparse matrix"""
  row=[]
  col=[]
  values=[]
  if isinstance(dataset,(list)):
    for row_num,sent in enumerate(tqdm(dataset)):
      word_freq = dict(Counter(sent.split()))
      for word,word_freq in word_freq.items():
        if len(word)<2:
          continue
        list_idx = vocab.get(word,-1)
        if list_idx != -1:
          row.append(row_num)
          col.append(list_idx)
          values.append(word_freq)
    return csr_matrix((values, (row,col)), shape=(len(dataset),len(vocab)))
  else:
    return 'pass a list'

In [46]:
cc=list(df['cleaned_text'])
sp_mat = transform(cc,vocab).toarray()

100%|██████████| 10/10 [00:00<00:00, 16301.22it/s]


In [39]:
strings = ["the method of lagrange multipliers is the economists workhorse for solving optimization problems",
           "the technique is a centerpiece of economic theory but unfortunately its usually taught poorly"]
vocab = fit_1(strings)
print(list(vocab.keys()))
print(transform(strings, vocab).toarray())

['but', 'centerpiece', 'economic', 'economists', 'for', 'is', 'its', 'lagrange', 'method', 'multipliers', 'of', 'optimization', 'poorly', 'problems', 'solving', 'taught', 'technique', 'the', 'theory', 'unfortunately', 'usually', 'workhorse']
[[0 0 0 1 1 1 0 1 1 1 1 1 0 1 1 0 0 2 0 0 0 1]
 [1 1 1 0 0 1 1 0 0 0 1 0 1 0 0 1 1 1 1 1 1 0]]


In [42]:
sp_mat

array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)